In [34]:
import polars as pl
import yfinance as yf
from datetime import datetime, timedelta
from tqdm import tqdm
import pandas as pd

# List of top 100 S&P 500 companies
sp500_tickers = [
    "AAPL", "MSFT", "AMZN", "NVDA", "GOOGL", "GOOG", "META", "BRK.B", "TSLA", "UNH",
    "LLY", "JNJ", "V", "XOM", "WMT", "JPM", "PG", "MA", "CVX", "MRK",
    "AVGO", "HD", "ABBV", "KO", "PEP", "COST", "BAC", "ADBE", "TMO", "CSCO",
    "PFE", "MCD", "ACN", "NFLX", "ABT", "CRM", "DHR", "LIN", "ORCL", "VZ",
    "NKE", "DIS", "INTC", "WFC", "TXN", "NEE", "PM", "BMY", "AMGN", "UNP",
    "MS", "AMD", "HON", "LOW", "INTU", "QCOM", "RTX", "CAT", "GS", "GE",
    "UPS", "BLK", "SPGI", "SCHW", "IBM", "PLD", "AMAT", "TGT", "ISRG", "MDT",
    "AXP", "SYK", "LMT", "ADP", "NOW", "BKNG", "CB", "CVS", "DE", "C",
    "ELV", "AMT", "TJX", "MO", "GILD", "ZTS", "CI", "USB", "MMC", "ADI",
    "PGR", "REGN", "DUK", "SO", "EQIX", "FDX", "GM", "BDX", "APD", "EMR",
    "ETN", "EOG", "VRTX", "TFC", "AON", "MET", "SLB", "NSC", "CSX", "COF"
]

def get_sp500_data():
    """
    Function to get quarterly price and dividend data for top 100 S&P 500 companies
    Returns data in CSV format using Polars for efficient processing
    """
    # Calculate date ranges
    end_date = datetime.now()
    start_date = end_date - timedelta(days=3*365)  # 3 years of data
    
    # Initialize list to store data
    all_data = []
    failed_tickers = []
    
    # Create progress bar
    pbar = tqdm(sp500_tickers, desc="Processing companies")
    
    for symbol in pbar:
        try:
            pbar.set_description(f"Processing {symbol}")
            
            # Download data using yfinance
            stock = yf.Ticker(symbol)
            
            # Get quarterly price data
            prices = stock.history(start=start_date, end=end_date, interval='3mo')
            
            # Skip if no price data
            if prices.empty:
                pbar.write(f"No price data available for {symbol}")
                failed_tickers.append((symbol, "No price data"))
                continue
            
            # Get dividend data
            dividends = stock.dividends
            
            # Convert to polars DataFrame for faster processing
            prices_pl = pl.from_pandas(prices)
            dividends_pl = pl.from_pandas(dividends.reset_index()) if not dividends.empty else pl.DataFrame()
            
            # Process each quarter
            for idx in range(len(prices)):
                quarter_start = prices.index[idx]
                quarter_end = quarter_start + pd.Timedelta(days=92)
                
                # Calculate quarterly dividends using polars
                if not dividends_pl.is_empty():
                    quarter_dividends = dividends_pl.filter(
                        (pl.col('Date') >= quarter_start) & 
                        (pl.col('Date') < quarter_end)
                    ).select(pl.col('Dividends').sum()).item()
                else:
                    quarter_dividends = 0.0
                
                # Add to dataset
                all_data.append({
                    'Symbol': symbol,
                    'Year': quarter_start.year,
                    'Quarter': quarter_start.quarter,
                    'Open': round(float(prices.iloc[idx]['Open']), 2),
                    'High': round(float(prices.iloc[idx]['High']), 2),
                    'Low': round(float(prices.iloc[idx]['Low']), 2),
                    'Close': round(float(prices.iloc[idx]['Close']), 2),
                    'Dividends': round(float(quarter_dividends), 4)
                })
                
        except Exception as e:
            error_msg = str(e)
            pbar.write(f"Error processing {symbol}: {error_msg}")
            failed_tickers.append((symbol, error_msg))
            continue
    
    pbar.close()
    
    # Print summary of failed tickers
    if failed_tickers:
        print("\nFailed to process the following tickers:")
        for ticker, error in failed_tickers:
            print(f"{ticker}: {error}")
            
    # Convert to Polars DataFrame
    df = pl.DataFrame(all_data)
    
    # Sort data
    df = df.sort(["Symbol", "Year", "Quarter"])
    
    # Return CSV string
    return df.write_csv(return_string=True)


SystemError: <class 'numpy.iinfo'> returned a result with an exception set

{'quota': {'used': 0, 'remaining': 25000, 'resets': '2025-02-21T23:00:00Z'}}